In [24]:
%load_ext autoreload
%autoreload 2

import os
os.environ['CUDA_VISIBLE_DEVICES']=''

import sys;
# path to t5-experiments repo
sys.path.append('..')

In [1]:
import torch
import pandas as pd

In [10]:
def get_source(alpha: int, beta: int, gamma: int, no_roots: bool) -> list:

    if beta > 0 and alpha > 0 and gamma > 0:
        source = f"{alpha}x^2+{beta}x+{gamma}=0"
    elif beta > 0 and alpha < 0 and gamma > 0:
         source = f"{alpha}x^2+{beta}x+{gamma}=0"
    elif beta > 0 and alpha < 0 and gamma < 0:
        source = f"{alpha}x^2+{beta}x{gamma}=0"   
    elif beta > 0 and alpha > 0 and gamma < 0:
         source = f"{alpha}x^2+{beta}x{gamma}=0" 
    elif beta < 0 and alpha > 0 and gamma > 0:
        source = f"{alpha}x^2{beta}x+{gamma}=0"
    elif beta < 0 and alpha < 0 and gamma > 0:
        source = f"{alpha}x^2{beta}x+{gamma}=0"
    elif beta < 0 and alpha > 0 and gamma < 0:
        source = f"{alpha}x^2{beta}x{gamma}=0"
    else:
        source = f"{alpha}x^2{beta}x{gamma}=0"
    
    if gamma == 1 and beta > 0 and gamma > 0 :
        source = f"x^2+{beta}x+{gamma}=0"
    elif gamma == 1 and beta > 0 and gamma < 0 :
        source = f"x^2+{beta}x{gamma}=0"
    elif gamma == 1 and beta < 0 and gamma > 0 :
        source = f"x^2{beta}x+{gamma}=0"
    elif gamma == 1 and beta < 0 and gamma < 0 :
        source = f"x^2{beta}x{gamma}=0"
    
    if no_roots == False:
        D = float(beta*beta - 4*alpha*gamma)

    if alpha == beta and no_roots == False:
        if beta > 0:
            solution = f"D=({beta}^2-4*{alpha}*{gamma})={D};x=(-{beta}/(2*({alpha}))={float(-beta/(2*alpha))};x=(-{beta}/(2*({alpha}))={float(-beta/(2*alpha))}"
            answer = f"x={float(-beta/(2*alpha))};"
            equation = f"{source};{solution};{answer}"
        elif beta < 0:
            solution = f"D=({beta}^2-4*{alpha}*{gamma};x=({(-1)*beta}/(2*({alpha}))={float(-beta/(2*alpha))};x=({(-1)*beta}/(2*({alpha}))={float(-beta/(2*alpha))}"
            answer = f"x={float(-beta/(2*alpha))};"
            equation = f"{source};{solution};{answer}"
    elif alpha != beta and no_roots == False:
        if beta > 0:
            solution = f"D=({beta}^2-4*{alpha}*{gamma})={D};x=(-{beta}-{np.sqrt(D)})/(2*{alpha})={float((-beta-np.sqrt(D))/(2*alpha))}; x=(-{beta}+{np.sqrt(D)})/(2*{alpha})={float((-beta+np.sqrt(D))/(2*alpha))}"
            answer = f"x={float((-beta-np.sqrt(D))/(2*alpha))};x={float((-beta+np.sqrt(D))/(2*alpha))};"
            equation = f"{source};{solution};{answer}"
        elif beta < 0:
            solution = f"D=({beta}^2-4*{alpha}*{gamma})={D};x=({(-1)*beta}-{np.sqrt(D)})/(2*{alpha})={(-beta-np.sqrt(D))/(2*alpha)}; x=({beta}+{np.sqrt(D)})/(2*{alpha})={(-beta+np.sqrt(D))/(2*alpha)}"
            answer = f"x={(-beta-np.sqrt(D))/(2*alpha)};x={(-beta+np.sqrt(D))/(2*alpha)};"
            equation = f"{source};{solution};{answer}"
    if no_roots:
        #source = f"{alpha}x^2+{beta}x+{gamma}=0"
        solution = f"D=({beta}^2-4*{alpha}*{gamma})={beta*beta - 4*alpha*gamma}"
        answer = f"x=nan;"
        equation = f"{source};{solution};{answer}"

    return equation

In [11]:
def get_roots(number_of_samples_to_add: int, one_root: bool, two_roots:bool, no_roots: bool) -> pd.DataFrame: 
    start_range = -1000
    end_range = 1000
    start_range_coeff = 1
    end_range_coeff = 10
    coeffs = []
    range_coeff = [i for i in range(start_range, end_range + 1) if i != 0]
    multiply_coeff = [i for i in range(start_range_coeff, end_range_coeff + 1)]
    a = np.random.choice(range_coeff, number_of_samples_to_add)
    b = np.random.choice(range_coeff, number_of_samples_to_add)
    intercepts = np.random.choice(range_coeff, number_of_samples_to_add)
    coeff = np.random.choice(multiply_coeff, number_of_samples_to_add)
    if number_of_samples_to_add >= 0:
        for root_1, root_2, intercept, ksi in zip(a, b, intercepts, coeff):
            if root_1 == root_2:
                if one_root:
                    alpha = ksi 
                    beta = -2*ksi*root_1
                    gamma = ksi*root_1*root_1
                    coeffs.append(get_source(alpha, beta, gamma, no_roots=False))
            elif root_1 != root_2 and root_1 != (-1)*root_2:
                if two_roots:
                    alpha = ksi
                    beta = -ksi*(root_1 + root_2)
                    gamma = ksi*root_1*root_2
                    coeffs.append(get_source(alpha, beta, gamma, no_roots=False))
                if no_roots:
                    D = root_2*root_2 - 4*root_1*intercept
                    if D < 0:
                        coeffs.append(get_source(root_1, root_2, intercept, no_roots=True))
                    else:
                        continue
    print(coeffs, "\n")
    dataset_roots = pd.DataFrame(coeffs, columns=['forms'])
    return dataset_roots



In [12]:
df_no_roots = get_roots(number_of_samples_to_add = 55500, one_root=False, two_roots=False, no_roots=True)
df_no_roots #20 000

['-806x^2+82x-354=0;D=(82^2-4*-806*-354)=-1134572;x=nan;', '710x^2-189x+952=0;D=(-189^2-4*710*952)=-2667959;x=nan;', '-511x^2+899x-672=0;D=(899^2-4*-511*-672)=-565367;x=nan;', '-259x^2+168x-105=0;D=(168^2-4*-259*-105)=-80556;x=nan;', '-593x^2-844x-853=0;D=(-844^2-4*-593*-853)=-1310980;x=nan;', '-477x^2+616x-283=0;D=(616^2-4*-477*-283)=-160508;x=nan;', '-939x^2+604x-915=0;D=(604^2-4*-939*-915)=-3071924;x=nan;', '497x^2+271x+344=0;D=(271^2-4*497*344)=-610431;x=nan;', '-849x^2-483x-196=0;D=(-483^2-4*-849*-196)=-432327;x=nan;', '-417x^2-505x-863=0;D=(-505^2-4*-417*-863)=-1184459;x=nan;', '-684x^2+700x-588=0;D=(700^2-4*-684*-588)=-1118768;x=nan;', '144x^2-141x+364=0;D=(-141^2-4*144*364)=-189783;x=nan;', '-342x^2-453x-248=0;D=(-453^2-4*-342*-248)=-134055;x=nan;', '58x^2+249x+447=0;D=(249^2-4*58*447)=-41703;x=nan;', '752x^2-962x+618=0;D=(-962^2-4*752*618)=-933500;x=nan;', '859x^2-858x+869=0;D=(-858^2-4*859*869)=-2249720;x=nan;', '-534x^2-889x-740=0;D=(-889^2-4*-534*-740)=-790319;x=nan;', '-65

,forms
0,-806x^2+82x-354=0;D=(82^2-4*-806*-354)=-113457...
1,710x^2-189x+952=0;D=(-189^2-4*710*952)=-266795...
2,-511x^2+899x-672=0;D=(899^2-4*-511*-672)=-5653...
3,-259x^2+168x-105=0;D=(168^2-4*-259*-105)=-8055...
4,-593x^2-844x-853=0;D=(-844^2-4*-593*-853)=-131...
...,...
20590,499x^2-459x+229=0;D=(-459^2-4*499*229)=-246403...
20591,583x^2+211x+729=0;D=(211^2-4*583*729)=-1655507...
20592,367x^2+719x+810=0;D=(719^2-4*367*810)=-672119;...
20593,789x^2-555x+599=0;D=(-555^2-4*789*599)=-158241...


In [13]:
df_one_roots = get_roots(number_of_samples_to_add = 59000000, one_root=True, two_roots=False, no_roots=False)
df_one_roots #30 000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,forms
0,10x^2-14540x+5285290=0;D=(-14540^2-4*10*528529...
1,3x^2+1884x+295788=0;D=(1884^2-4*3*295788)=0.0;...
2,5x^2-5930x+1758245=0;D=(-5930^2-4*5*1758245)=0...
3,6x^2+10488x+4583256=0;D=(10488^2-4*6*4583256)=...
4,4x^2-16x+16=0;D=(-16^2-4*4*16)=0.0;x=(16-0.0)/...
...,...
29468,6x^2+11388x+5403606=0;D=(11388^2-4*6*5403606)=...
29469,8x^2+3056x+291848=0;D=(3056^2-4*8*291848)=0.0;...
29470,5x^2-3480x+605520=0;D=(-3480^2-4*5*605520)=0.0...
29471,8x^2+9744x+2967048=0;D=(9744^2-4*8*2967048)=0....


In [14]:
df_two_roots = get_roots(number_of_samples_to_add = 50100, one_root=False, two_roots=True, no_roots=False)
df_two_roots #50 000

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,forms
0,2x^2+904x+30710=0;D=(904^2-4*2*30710)=571536.0...
1,8x^2-1280x-4460808=0;D=(-1280^2-4*8*-4460808)=...
2,1x^2+1072x+104112=0;D=(1072^2-4*1*104112)=7327...
3,5x^2-5080x+322320=0;D=(-5080^2-4*5*322320)=193...
4,8x^2+5816x-789984=0;D=(5816^2-4*8*-789984)=591...
...,...
50031,8x^2-9424x+2504520=0;D=(-9424^2-4*8*2504520)=8...
50032,3x^2+3861x+1239108=0;D=(3861^2-4*3*1239108)=38...
50033,4x^2-1024x-712388=0;D=(-1024^2-4*4*-712388)=12...
50034,6x^2+9870x+3900600=0;D=(9870^2-4*6*3900600)=38...


In [15]:
def get_dataset(df: pd.DataFrame):
    df.to_csv('root.csv')
    train = pd.read_csv(r'./root.csv', sep=",")
    r = train.forms.str.split(';', expand=True)
    print(r)
    r["sol"] = r[1] + r[2] + r[3] + r[4] + r[5]
    r['sol'] = r[[1, 2, 3, 4, 5]].agg(';'.join, axis=1)
    df_root = r.drop(columns=[1, 2, 3, 4, 5, 6])
    
    return df_root

In [16]:
def get_dataset4no_roots(df: pd.DataFrame):
    df.to_csv('root.csv')
    train = pd.read_csv(r'./root.csv', sep=",")
    r = train.forms.str.split(';', expand=True)
    print(r)
    r["sol"] = r[1] + r[2] 
    r['sol'] = r[[1, 2]].agg(';'.join, axis=1)
    df_root = r.drop(columns=[1, 2, 3])
    
    return df_root

In [17]:
df_one = get_dataset(df_one_roots)
df_two = get_dataset(df_two_roots)
df_no = get_dataset4no_roots(df_no_roots)

                            0                              1  \
0      10x^2-14540x+5285290=0  D=(-14540^2-4*10*5285290)=0.0   
1         3x^2+1884x+295788=0      D=(1884^2-4*3*295788)=0.0   
2        5x^2-5930x+1758245=0    D=(-5930^2-4*5*1758245)=0.0   
3       6x^2+10488x+4583256=0    D=(10488^2-4*6*4583256)=0.0   
4               4x^2-16x+16=0           D=(-16^2-4*4*16)=0.0   
...                       ...                            ...   
29468   6x^2+11388x+5403606=0    D=(11388^2-4*6*5403606)=0.0   
29469     8x^2+3056x+291848=0      D=(3056^2-4*8*291848)=0.0   
29470     5x^2-3480x+605520=0     D=(-3480^2-4*5*605520)=0.0   
29471    8x^2+9744x+2967048=0     D=(9744^2-4*8*2967048)=0.0   
29472   9x^2-16164x+7257636=0   D=(-16164^2-4*9*7257636)=0.0   

                                 2                             3         4  \
0       x=(14540-0.0)/(2*10)=727.0   x=(-14540+0.0)/(2*10)=727.0   x=727.0   
1       x=(-1884-0.0)/(2*3)=-314.0    x=(-1884+0.0)/(2*3)=-314.0  x=-314.0 

In [21]:
df_one = df_one.rename(columns={0: "input", "sol": "output"})
df_two = df_two.rename(columns={0: "input", "sol": "output"})
df_no = df_no.rename(columns={0: "input", "sol": "output"})

In [22]:
frames = [df_one, df_two, df_no]

result = pd.concat(frames, ignore_index=True)

In [23]:
result

,input,output
0,10x^2-14540x+5285290=0,D=(-14540^2-4*10*5285290)=0.0;x=(14540-0.0)/(2...
1,3x^2+1884x+295788=0,D=(1884^2-4*3*295788)=0.0;x=(-1884-0.0)/(2*3)=...
2,5x^2-5930x+1758245=0,D=(-5930^2-4*5*1758245)=0.0;x=(5930-0.0)/(2*5)...
3,6x^2+10488x+4583256=0,D=(10488^2-4*6*4583256)=0.0;x=(-10488-0.0)/(2*...
4,4x^2-16x+16=0,D=(-16^2-4*4*16)=0.0;x=(16-0.0)/(2*4)=2.0; x=(...
...,...,...
100099,499x^2-459x+229=0,D=(-459^2-4*499*229)=-246403;x=nan
100100,583x^2+211x+729=0,D=(211^2-4*583*729)=-1655507;x=nan
100101,367x^2+719x+810=0,D=(719^2-4*367*810)=-672119;x=nan
100102,789x^2-555x+599=0,D=(-555^2-4*789*599)=-1582419;x=nan


In [25]:
#shuffle data
df_raw = result.sample(frac=1).reset_index(drop=True)

In [27]:
train_dataset = df_raw[0:80000]
test_dataset = df_raw[80000:]

In [28]:
train_dataset.to_csv('dataset_train.csv')
test_dataset.to_csv('dataset_test.csv')